<a href="https://colab.research.google.com/github/HaykTarkhanyan/python_math_ml_course/blob/main/%5BInitial%5D_Python_%7C_%D5%8F4_%7C_For%2C_while_%D6%81%D5%AB%D5%AF%D5%AC%D5%A5%D6%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> (ToDo)

# Hyperparameter tuning

## Train Val Test split

Role of Each Split
1. **Training Set:** Used to fit or train your model parameters (weights, decision tree splits, etc.).
2. **Validation Set:** Used to tune hyperparameters or compare different models. It provides feedback on how different settings affect performance before touching the test set.
3. **Test Set:** Used **once** at the end to get an unbiased estimate of the final chosen model’s performance.

The same way we need a dataset separate from the training set to tune hyperparameters, we need a separate dataset to evaluate the model’s final performance. This is the test set. 

### Data Prep

In [1]:
import os
import pandas as pd 
import plotly.express as px

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [43]:
if "House_Rent_Dataset.csv" in os.listdir():
    path = "House_Rent_Dataset.csv"
else:
    import kagglehub
    path = kagglehub.dataset_download("iamsouravbanerjee/house-rent-prediction-dataset")
    path = os.path.join(path, "House_Rent_Dataset.csv")
    
df = pd.read_csv(path)

In [ ]:
target = "Rent"

# Split into train val, test
train_size = 0.7

X = df.drop(columns=[target])
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=train_size, random_state=5090)
# X_train -> 70%, X_test -> 30%

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, 
                                                  test_size=0.5, random_state=5090)

# 100
# 70 30
# 30 -> 15, 15

print(f"Sizes: Train={len(X_train)}, Val={len(X_val)}, Test={len(X_test)}")

Sizes: Train=3322, Val=712, Test=712


**Note:** It would have been better to do cross-validation here

In [46]:
COLS_TO_DROP = ["Posted On", "Floor", "Area Locality"]

def basic_preprocessing(df):
    df = df.drop(columns=COLS_TO_DROP)
    df.rename(columns={"Furnishing Status": "Furnish"}, inplace=True)
    return df

In [47]:
X_train = basic_preprocessing(X_train)
X_val = basic_preprocessing(X_val)
X_test = basic_preprocessing(X_test)

X_val

,BHK,Size,Area Type,City,Furnish,Tenant Preferred,Bathroom,Point of Contact
284,2,900,Super Area,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
257,2,450,Carpet Area,Kolkata,Unfurnished,Bachelors,1,Contact Owner
872,2,400,Carpet Area,Mumbai,Furnished,Bachelors/Family,1,Contact Owner
3749,3,1610,Super Area,Chennai,Semi-Furnished,Family,3,Contact Agent
2732,2,1000,Super Area,Delhi,Semi-Furnished,Family,1,Contact Owner
...,...,...,...,...,...,...,...,...
2357,2,850,Super Area,Bangalore,Semi-Furnished,Bachelors/Family,2,Contact Owner
3842,2,1000,Carpet Area,Chennai,Unfurnished,Bachelors,2,Contact Owner
1799,1,500,Super Area,Bangalore,Unfurnished,Bachelors/Family,1,Contact Owner
3931,2,1800,Super Area,Hyderabad,Furnished,Bachelors/Family,2,Contact Owner


In [ ]:
All - 1, 5, 10, 20  

Normalize -> 
- min -> 1 
- max -> 20

(x - 1) / (20 - 1)

Train / test -> 1, 5, 10 | 20 

min -> 1
max -> 10

(x - 1) / (10 - 1)

(x - a) / (b - a)

-----
(20 - 1) / (10 - 1) -> 1.9


In [48]:
COLS_OHE = ["City"]
COLS_ORDINAL = ["Furnish"]
COLS_STANDARD_SCALE = ["BHK", "Bathroom", "Size"]

In [49]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, \
    PolynomialFeatures

ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore", drop="first")
ordinal = OrdinalEncoder(categories=[["Unfurnished", "Semi-Furnished", "Furnished"]])
standard = StandardScaler()
poly = PolynomialFeatures(degree=3)

FIT ONLY ON TRAIN, OTHERWISE DATA LEAKAGE


In [50]:
ohe.fit(X_train[COLS_OHE])
ordinal.fit(X_train[COLS_ORDINAL])
poly.fit(X_train[COLS_STANDARD_SCALE])

X_train_poly = poly.transform(X_train[COLS_STANDARD_SCALE])

standard.fit(X_train_poly)

StandardScaler()

In [51]:
def preprocess_data(X, ohe, ordinal, standard):
    X_ohe = ohe.transform(X[COLS_OHE])
    X_ordinal = ordinal.transform(X[COLS_ORDINAL])
    x_poly = poly.transform(X[COLS_STANDARD_SCALE])
    X_poly_standard = standard.transform(x_poly)

    X_poly_standard = pd.DataFrame(X_poly_standard, columns=poly.get_feature_names_out())

    X_ohe = pd.DataFrame(X_ohe, columns=ohe.get_feature_names_out(COLS_OHE))
    X_ordinal = pd.DataFrame(X_ordinal, columns=COLS_ORDINAL)
    X_poly_standard = pd.DataFrame(X_poly_standard, columns=poly.get_feature_names_out(COLS_STANDARD_SCALE))
    
    X = pd.concat([X_poly_standard, X_ohe, X_ordinal], axis=1)
    return X

In [52]:
X_train = preprocess_data(X_train, ohe, ordinal, standard)
X_val = preprocess_data(X_val, ohe, ordinal, standard)
X_test = preprocess_data(X_test, ohe, ordinal, standard)

X_train

,1,BHK,Bathroom,Size,BHK^2,BHK Bathroom,BHK Size,Bathroom^2,Bathroom Size,Size^2,...,Bathroom^3,Bathroom^2 Size,Bathroom Size^2,Size^3,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Furnish
0,0.0,1.110492,1.189251,0.829344,0.999475,1.114653,0.806053,0.928818,0.707418,0.344902,...,0.506051,0.356248,0.175618,0.041691,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,-0.097846,-1.091749,-0.563145,-0.254682,-0.677640,-0.450178,-0.759804,-0.549198,-0.376971,...,-0.427136,-0.317066,-0.237390,-0.183322,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,-0.097846,0.048751,-0.408424,-0.254682,-0.165556,-0.374043,-0.126570,-0.291431,-0.327319,...,-0.175894,-0.202237,-0.197317,-0.174276,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-0.097846,0.048751,0.133100,-0.254682,-0.165556,-0.107570,-0.126570,-0.065884,-0.093378,...,-0.175894,-0.129164,-0.118141,-0.116251,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,-0.097846,-1.091749,-0.021621,-0.254682,-0.677640,-0.183705,-0.759804,-0.436425,-0.169767,...,-0.427136,-0.298798,-0.202326,-0.137638,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3317,0.0,-0.097846,0.048751,0.055739,-0.254682,-0.165556,-0.145637,-0.126570,-0.098105,-0.132528,...,-0.175894,-0.139603,-0.131391,-0.127479,0.0,0.0,1.0,0.0,0.0,1.0
3318,0.0,-1.306183,-1.091749,-0.795227,-1.007177,-0.933682,-0.735686,-0.759804,-0.597530,-0.437127,...,-0.427136,-0.324895,-0.247570,-0.192217,0.0,1.0,0.0,0.0,0.0,0.0
3319,0.0,-0.097846,0.048751,0.055739,-0.254682,-0.165556,-0.145637,-0.126570,-0.098105,-0.132528,...,-0.175894,-0.139603,-0.131391,-0.127479,0.0,1.0,0.0,0.0,0.0,1.0
3320,0.0,-1.306183,0.048751,-0.717866,-1.007177,-0.677640,-0.716652,-0.126570,-0.420314,-0.418985,...,-0.175894,-0.243993,-0.228341,-0.189804,0.0,0.0,0.0,0.0,1.0,2.0


## Hyperparameter Tuning with Random Search and Grid Search


In [53]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import Lasso

# ConvergenceWarning
# ignore the warning
# from warnings import simplefilter
# simplefilter("ignore")


In [ ]:
learning_rate - 0.1, 1, 10
lambda - 0.4, 0.8 




Notes:
- confusing terminology, the regularization parameter is called alpha in sklearn
- lambda (alpha) is usually searched on a log scale


In [54]:
alphas = np.logspace(1, 3, 20)
print(alphas)

[  10.           12.74274986   16.23776739   20.69138081   26.36650899
   33.59818286   42.81332399   54.55594781   69.51927962   88.58667904
  112.88378917  143.84498883  183.29807108  233.57214691  297.63514416
  379.26901907  483.29302386  615.84821107  784.75997035 1000.        ]


In [17]:
10 ** -1

0.1

### Grid Search

In [55]:
# extend the grid 
param_grid = {
    "alpha": [10 ** i for i in range(-3, 4)],
    'max_iter': [10, 100, 50_000]
}

# get Decartian product of all the hyperparameters
param_grid

{'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'max_iter': [10, 100, 50000]}

In [56]:

res_list = []
for lamb in param_grid['alpha']:
    for max_iter in param_grid['max_iter']:
        model = Lasso(alpha=lamb, max_iter=max_iter)
        model.fit(X_train, y_train)
        
        y_pred_train = model.predict(X_train)
        y_pred_val = model.predict(X_val)
        
        # r2
        r2_train = model.score(X_train, y_train)
        r2_val = model.score(X_val, y_val)
        
        res = {
            "lambda": lamb,
            "max_iter": max_iter,
            "r2_train": r2_train,
            "r2_val": r2_val
        }
        
        print(res)
        res_list.append(res)
        


c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.386e+12, tolerance: 2.350e+09

c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.275e+12, tolerance: 2.350e+09



{'lambda': 0.001, 'max_iter': 10, 'r2_train': 0.286300900903835, 'r2_val': 0.5469368390683846}
{'lambda': 0.001, 'max_iter': 100, 'r2_train': 0.2957594907787815, 'r2_val': 0.5626080457488305}


c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.228e+12, tolerance: 2.350e+09

c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.386e+12, tolerance: 2.350e+09

c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.274e+12, tolerance: 2.350e+09



{'lambda': 0.001, 'max_iter': 50000, 'r2_train': 0.30240718968076064, 'r2_val': 0.5734121131426642}
{'lambda': 0.01, 'max_iter': 10, 'r2_train': 0.28630087985195296, 'r2_val': 0.5469369298770144}
{'lambda': 0.01, 'max_iter': 100, 'r2_train': 0.2957593913671396, 'r2_val': 0.5626080842528678}


c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.478e+11, tolerance: 2.350e+09

c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.385e+12, tolerance: 2.350e+09

c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.268e+12, tolerance: 2.350e+09



...

c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.302e+12, tolerance: 2.350e+09

c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.608e+12, tolerance: 2.350e+09



{'lambda': 10, 'max_iter': 50000, 'r2_train': 0.3019441918919157, 'r2_val': 0.5716460477070587}
{'lambda': 100, 'max_iter': 10, 'r2_train': 0.2859912802103456, 'r2_val': 0.5478256953197767}
{'lambda': 100, 'max_iter': 100, 'r2_train': 0.29462512075138036, 'r2_val': 0.5636114381784972}
{'lambda': 100, 'max_iter': 50000, 'r2_train': 0.29711504164066593, 'r2_val': 0.5664434496196209}
{'lambda': 1000, 'max_iter': 10, 'r2_train': 0.2822369767509024, 'r2_val': 0.5519400187789713}


c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.538e+12, tolerance: 2.350e+09

c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.189e+12, tolerance: 2.350e+09

c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.825e+10, tolerance: 2.350e+09



{'lambda': 1000, 'max_iter': 100, 'r2_train': 0.2826332175940921, 'r2_val': 0.555507368159095}
{'lambda': 1000, 'max_iter': 50000, 'r2_train': 0.28265746301940353, 'r2_val': 0.5563976587973434}


c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.739e+10, tolerance: 2.350e+09



In [57]:
res_df = pd.DataFrame(res_list)
res_df

,lambda,max_iter,r2_train,r2_val
0,0.001,10,0.286301,0.546937
1,0.001,100,0.295759,0.562608
2,0.001,50000,0.302407,0.573412
3,0.010,10,0.286301,0.546937
4,0.010,100,0.295759,0.562608
5,0.010,50000,0.302407,0.573413
6,0.100,10,0.286301,0.546938
7,0.100,100,0.295758,0.562608
8,0.100,50000,0.302407,0.573422
9,1.000,10,0.286299,0.546947


In [21]:
px.scatter(res_df, x="lambda", y="max_iter", log_x=True, log_y=True, color="r2_val", title="Grid Search")

## Grid Search with sklearn

In [58]:
param_grid = {
    "alpha": [10 ** i for i in range(-3, 2)],
    'max_iter': [10, 100, 50_000], 
    'tol': [1e-4,  1e-2],
    "fit_intercept": [True, False]  
}

In [ ]:
grid_search = GridSearchCV(
    estimator=Lasso(),
    param_grid=param_grid,
    scoring='r2',       # Optimize R^2
    cv=3,               # 3-fold cross-validation
    n_jobs=-1,          # Use all CPU cores
    verbose=True        # Print more information  
)

X_use = pd.concat([X_train, X_val]).head(1_000)
y_use = pd.concat([y_train, y_val]).head(1_000)

grid_search.fit(X_use, y_use)


Fitting 3 folds for each of 60 candidates, totalling 180 fits


GridSearchCV(cv=3, estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 0.1, 1, 10],
                         'fit_intercept': [True, False],
                         'max_iter': [10, 100, 50000], 'tol': [0.0001, 0.01]},
             scoring='r2', verbose=True)

In [59]:
print("\n== Grid Search Results ==")
print("Best parameters found:", grid_search.best_params_)
print("Best cross-validation R^2:", grid_search.best_score_)



== Grid Search Results ==
Best parameters found: {'alpha': 10, 'fit_intercept': True, 'max_iter': 50000, 'tol': 0.01}
Best cross-validation R^2: 0.5424913981816509


In [60]:
from sklearn.metrics import r2_score, root_mean_squared_error

best_lasso = grid_search.best_estimator_

y_train_pred = best_lasso.predict(X_train)
y_val_pred = best_lasso.predict(X_val)
y_test_pred = best_lasso.predict(X_test)

def evaluate_model(y_true, y_pred):
    mse = root_mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mse, r2

mse_train, r2_train = evaluate_model(y_train, y_train_pred)
mse_val, r2_val = evaluate_model(y_val, y_val_pred)
mse_test, r2_test = evaluate_model(y_test, y_test_pred)

print(f"Train: RMSE={mse_train:.2f}, R^2={r2_train:.2f}")
print(f"Val: RMSE={mse_val:.2f}, R^2={r2_val:.2f}")
print(f"Test: RMSE={mse_test:.2f}, R^2={r2_test:.2f}")


Train: RMSE=83108.20, R^2=0.02
Val: RMSE=44000.30, R^2=0.58
Test: RMSE=47140.67, R^2=0.27


## Random Search

In [61]:
from scipy.stats import loguniform, randint


In [63]:
param_dist = {
    'alpha': loguniform(1e-3, 1e4),  # alpha in [1e-3, 1e4]
    'max_iter': randint(10, 5000)    # integer in [10, 5000)
}

print("Random Search will sample from these hyperparameter distributions:")
for p, dist in param_dist.items():
    print(f"  {p}: {dist}")

Random Search will sample from these hyperparameter distributions:
  alpha: <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025048DF6770>
  max_iter: <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025050F516F0>


In [64]:
random_search = RandomizedSearchCV(
    estimator=Lasso(),
    param_distributions=param_dist,
    n_iter=100,         # try 100 combinations of hyperparameters
    scoring='r2',       # optimize R^2
    cv=3,               # 3-fold cross-validation
    n_jobs=-1,          # use all CPU cores
    verbose=1,          # get more information
    random_state=509   # random seed for reproducibility
)

In [30]:
X_use = pd.concat([X_train, X_val]).head(1_000)
y_use = pd.concat([y_train, y_val]).head(1_000)


random_search.fit(X_use, y_use)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


c:\Users\hayk_\.conda\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.937e+10, tolerance: 2.955e+08



RandomizedSearchCV(cv=3, estimator=Lasso(), n_iter=100, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000025037FF8E80>,
                                        'max_iter': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025037FF97E0>},
                   random_state=509, scoring='r2', verbose=1)

In [31]:

print("\n=== RandomizedSearchCV Results ===")
print("Best params:", random_search.best_params_)
print("Best CV R^2:", random_search.best_score_)


=== RandomizedSearchCV Results ===
Best params: {'alpha': 11.87976242665855, 'max_iter': 4585}
Best CV R^2: 0.5420305853566706


In [32]:
random_search.cv_results_

{'mean_fit_time': array([0.00711656, 0.00800848, 0.01639883, 0.00493169, 0.03920269,
        0.01878921, 0.03078715, 0.00602428, 0.06687085, 0.02392554,
        0.03888313, 0.05870867, 0.00405304, 0.01272337, 0.00949931,
        0.02375094, 0.05600119, 0.00656223, 0.07336688, 0.05129862,
        0.04841129, 0.01057466, 0.01479801, 0.01549991, 0.009715  ,
        0.07277314, 0.00604137, 0.00731683, 0.03629478, 0.00680598,
        0.03837021, 0.02040132, 0.04617818, 0.00693742, 0.00673501,
        0.02214535, 0.0070231 , 0.01767802, 0.02047157, 0.01068656,
        0.03303337, 0.02222919, 0.00713054, 0.05738052, 0.00551462,
        0.03851438, 0.04714799, 0.04711008, 0.02859553, 0.04198591,
        0.00535266, 0.05226008, 0.03640064, 0.00590746, 0.00492358,
        0.02960356, 0.05436889, 0.05619017, 0.04693087, 0.00601029,
        0.02669708, 0.00619006, 0.01373895, 0.01839383, 0.03812782,
        0.05356129, 0.02142739, 0.00402013, 0.07606483, 0.00584205,
        0.00483274, 0.04997969,

In [33]:
import pandas as pd

# Suppose you already have a fitted RandomizedSearchCV object named random_search
random_results_df = pd.DataFrame(random_search.cv_results_)

# Each hyperparameter is in a column with 'param_' prefix:
# e.g., 'param_alpha', 'param_max_iter', etc.
# You can display them along with the mean CV score, etc.
df_random_search = random_results_df[
    ['param_alpha', 'param_max_iter', 'mean_test_score', 'rank_test_score']
].sort_values(by='mean_test_score', ascending=False)

df_random_search

,param_alpha,param_max_iter,mean_test_score,rank_test_score
97,11.879762,4585,0.542031,1
81,8.479559,4185,0.541872,2
76,5.404961,3828,0.541591,3
32,11.671067,3690,0.541486,4
16,2.321346,4282,0.541280,5
...,...,...,...,...
17,5252.818689,4238,0.450202,96
67,5642.591616,2679,0.438345,97
90,6091.199063,2999,0.423569,98
39,9017.145603,1777,0.342540,99


In [34]:
px.scatter(df_random_search, x="param_alpha", y="param_max_iter", color="mean_test_score", 
           title="Random Search", log_x=True, log_y=True, color_continuous_scale="viridis")	

## Manual Hyperparameter Tuning

## Bayesian Optimization

Sklearn 

`!pip install scikit-optimize`

`from skopt import BayesSearchCV`

Hyperopt (hyperparameter optimization library); optuna

In [122]:
!pip install hyperopt

  Using cached future-1.0.0-py3-none-any.whl.metadata (4.0 kB)
   ---------------------------------------- 1.6/1.6 MB 2.3 MB/s eta 0:00:00
Using cached future-1.0.0-py3-none-any.whl (491 kB)
   ---------------------------------------- 203.0/203.0 kB 2.4 MB/s eta 0:00:00


In [35]:
from sklearn.model_selection import cross_val_score, train_test_split, KFold

In [65]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

In [66]:
# 1) Define the search space for alpha and max_iter
space = {
    'alpha': hp.loguniform('alpha', np.log(1e-3), np.log(1e3)),
    'max_iter': hp.randint('max_iter', 5000)  # integer in [0, 5000)
}

# 2) We’ll do 5-fold cross-validation to evaluate each set of params
cv = KFold(n_splits=5, shuffle=True, random_state=42)

def objective(params):
    # Extract hyperparameters
    alpha = params['alpha']
    max_iter = int(params['max_iter'])  # ensure integer
    
    # Create Lasso model
    model = Lasso(alpha=alpha, max_iter=max_iter, random_state=509)
    
    # Cross-validate (negative MSE, so we can minimize it)
    mse_scores = -cross_val_score(
        model, X_train, y_train,
        cv=cv, scoring='neg_mean_squared_error'
    )
    avg_mse = np.mean(mse_scores)
    
    # Hyperopt tries to minimize the returned value
    return {
        'loss': avg_mse,
        'status': STATUS_OK,
        # Optional: attach other info if you like
        'params': params
    }


In [ ]:
trials = Trials()  # To store results

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,       # Number of trials (increase for better search)
    trials=trials,
    rstate=np.random.default_rng(509)  # for reproducibility
)

print("\nBest hyperparameters found:")
print(best)


In [72]:
best

{'alpha': 82.26205843443854, 'max_iter': 3194}

In [68]:
best_alpha = best['alpha']
best_max_iter = int(best['max_iter'])

model_best = Lasso(alpha=best_alpha, max_iter=best_max_iter, random_state=42)
model_best.fit(X_train, y_train)

# Evaluate on the test set
y_pred_test = model_best.predict(X_test)

rmse, r2 = evaluate_model(y_test, y_pred_test)
print(f"Test set: RMSE={rmse:.2f}, R^2={r2:.2f}")

Test set: RMSE=37231.30, R^2=0.55


In [69]:
results_df = pd.DataFrame(trials.results)
results_df = results_df.sort_values(by='loss')

results_df["alpha"] = results_df["params"].apply(lambda x: x["alpha"])
results_df["max_iter"] = results_df["params"].apply(lambda x: x["max_iter"])
results_df


,loss,status,params,alpha,max_iter
14,5.158148e+09,ok,"{'alpha': 82.26205843443854, 'max_iter': 3194}",82.262058,3194
3,5.167013e+09,ok,"{'alpha': 23.927000650080377, 'max_iter': 206}",23.927001,206
13,5.290333e+09,ok,"{'alpha': 347.92671616536, 'max_iter': 1292}",347.926716,1292
11,5.358360e+09,ok,"{'alpha': 28.505108573753763, 'max_iter': 2196}",28.505109,2196
1,5.384187e+09,ok,"{'alpha': 0.013918988887753607, 'max_iter': 429}",0.013919,429
10,5.839504e+09,ok,"{'alpha': 0.0027521976271328466, 'max_iter': 792}",0.002752,792
16,6.432845e+09,ok,"{'alpha': 0.037966985847236734, 'max_iter': 1196}",0.037967,1196
8,6.677457e+09,ok,"{'alpha': 0.0014052766974551073, 'max_iter': 1...",0.001405,1355
6,7.403906e+09,ok,"{'alpha': 0.002452008538580964, 'max_iter': 1820}",0.002452,1820
18,7.584504e+09,ok,"{'alpha': 9.753940120503824, 'max_iter': 2707}",9.753940,2707


In [70]:
px.scatter(results_df, x="alpha", y="max_iter", title="Hyperopt Search",
              log_x=True)